In [1]:
%cd ../

/Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu


In [2]:
# If you get a SciPy error when installing Emukit, build it from source:

# git clone https://github.com/amzn/Emukit.git
# cd Emukit
# pip install -r requirements/requirements.txt
# python setup.py develop

In [3]:
import numpy as np
import pickle
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign, latin_design
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction
from emukit.core.loop import UserFunctionWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.core.optimization import GradientAcquisitionOptimizer

from sumo_grid_simulation.grid_simulation import Simulator
import experimental_design.config as config

### Pseudocode

Fitting a gaussian process to a simulator using Emukit takes the following form (taken from L48 lectures):

```
initialize GP with some randomly chosen points
while stopping condition is not met:
    compute candidate point(s) using GP and acquisition funciton (model_variance) -> new point
    evaluate this new point with our simulator/user function -> observation
    update model with new observation -> new GP
```

### Simulator

In [4]:
simulator = Simulator(end_time=300)

### User Function
 
This is the function we want to understand. Namedly, how are CO2 emissions and timeLoss (the time lost due to driving below the ideal speed) affected by the following parameters: 

```
gridSize: the size of the grid network where the simulation is carried out
edgeMaxSpeed: legal speed limit in m/s - this can be exceeded (11.11 == 40km/h)
maxSpeed: the absolute maximum velocity of any vehicle in m/s (55.55 == 200 km/h)
edgeLength: length of the roads between intersections in meters
numLanes: number of lanes per road
accel: The acceleration ability of vehicles in m/s^2.
```

*A complete list of parmeters analysed is discussed in our report.*

In [5]:
def user_function_time_loss(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average time loss / average route duration
        result.append(s['timeLoss']/s['duration'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

### Model Inputs / parameter space

In [6]:
parameter_space = config.get_parameter_space()

### Get init, test, and train points

Train points are used to optimize a GP without experimental design - to verify experimental design works.

In [7]:
design = RandomDesign(parameter_space)
init_X = design.get_samples(20)
init_Y = user_function_time_loss(init_X)

design = RandomDesign(parameter_space)
test_X = design.get_samples(50)
test_Y = user_function_time_loss(test_X)

design = RandomDesign(parameter_space)
train_X = design.get_samples(200)
train_Y = user_function_time_loss(train_X)

[[12.         13.2472251  16.00309042 44.15647269  1.          4.84037006]
 [ 3.         13.85991128 21.52428571 63.58881136  3.          4.53875461]
 [20.         20.49378826 13.58839796 56.78815434  1.          4.49833466]
 [12.         21.1110553  13.08387654 53.73658844  1.          3.32115863]
 [18.         11.63207666 18.40559721 38.82298417  3.          1.92153349]
 [ 6.         10.5960296   5.14359864 36.50388433  1.          3.45380455]
 [13.         24.10880585 16.08695157 48.01906136  2.          3.78158195]
 [15.         14.9719694  15.21440119 37.37005081  1.          1.62425583]
 [15.          8.60571641 25.98958494 56.45145224  2.          1.52055699]
 [18.         19.42708537  8.41937406 62.14267917  2.          3.62195194]
 [11.          9.17628712 14.01462276 58.14471293  1.          3.38775973]
 [ 5.          8.48478217 23.48863628 47.67340324  3.          3.32045529]
 [10.         15.25176756 13.1581459  64.54878793  1.          3.63920978]
 [ 6.         21.83984647


Evaluating input: 36 of 50

 Retrying in 1 seconds

Evaluating input: 37 of 50

 Retrying in 1 seconds

Evaluating input: 38 of 50

 Retrying in 1 seconds

Evaluating input: 39 of 50

 Retrying in 1 seconds

Evaluating input: 40 of 50

 Retrying in 1 seconds

Evaluating input: 41 of 50

 Retrying in 1 seconds

Evaluating input: 42 of 50

 Retrying in 1 seconds

Evaluating input: 43 of 50

 Retrying in 1 seconds

Evaluating input: 44 of 50

 Retrying in 1 seconds

Evaluating input: 45 of 50

 Retrying in 1 seconds

Evaluating input: 46 of 50

 Retrying in 1 seconds

Evaluating input: 47 of 50

 Retrying in 1 seconds

Evaluating input: 48 of 50

 Retrying in 1 seconds

Evaluating input: 49 of 50

 Retrying in 1 seconds

Evaluating input: 50 of 50

 Retrying in 1 seconds
[[16.         20.72584304  5.69355467 65.12260492  1.          3.28939768]
 [18.         20.73277493  5.06573506 39.19151712  1.          3.74252115]
 [13.         13.82300535  5.29443477 64.64586959  2.          2.42841


Evaluating input: 131 of 200

 Retrying in 1 seconds

Evaluating input: 132 of 200

 Retrying in 1 seconds

Evaluating input: 133 of 200

 Retrying in 1 seconds

Evaluating input: 134 of 200

 Retrying in 1 seconds

Evaluating input: 135 of 200

 Retrying in 1 seconds

Evaluating input: 136 of 200

 Retrying in 1 seconds

Evaluating input: 137 of 200

 Retrying in 1 seconds

Evaluating input: 138 of 200

 Retrying in 1 seconds

Evaluating input: 139 of 200

 Retrying in 1 seconds

Evaluating input: 140 of 200

 Retrying in 1 seconds

Evaluating input: 141 of 200

 Retrying in 1 seconds

Evaluating input: 142 of 200

 Retrying in 1 seconds

Evaluating input: 143 of 200

 Retrying in 1 seconds

Evaluating input: 144 of 200

 Retrying in 1 seconds

Evaluating input: 145 of 200

 Retrying in 1 seconds

Evaluating input: 146 of 200

 Retrying in 1 seconds

Evaluating input: 147 of 200

 Retrying in 1 seconds

Evaluating input: 148 of 200

 Retrying in 1 seconds

Evaluating input: 149 of 20

### Experimental Design - Model Variance

#### Model / Emulator (GP)

Our surrogate model is our emulator. In this case, a gaussian process.

In [8]:
emulator_ed = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_ed = GPyModelWrapper(emulator_ed, n_restarts=5)
emukit_model_ed.optimize()
emulator_ed

Optimization restart 1/5, f = -18.904667569713787
Optimization restart 2/5, f = 3.656207533743695
Optimization restart 3/5, f = 3.656206260245183
Optimization restart 4/5, f = 3.6562078363206005
Optimization restart 5/5, f = -16.417645787637227


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


GP_regression.,value,constraints,priors
rbf.variance,0.05627766991206398,+ve,
rbf.lengthscale,16.018276061971232,+ve,
Gaussian_noise.variance,1.0000006515337029e-10,+ve,


#### Acquisition Function

In [9]:
model_variance = ModelVariance(model=emukit_model_ed)

#### Optimizer

In [10]:
optimizer = GradientAcquisitionOptimizer(parameter_space)

#### Experimental Design Loop

In [11]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ed,
    space = parameter_space,
    acquisition = model_variance,
    acquisition_optimizer = optimizer,
)

In [12]:
ed_loop.run_loop(user_function_time_loss, 200)

Optimization restart 1/5, f = -18.904667569713787
Optimization restart 2/5, f = 3.6562047897258445
Optimization restart 3/5, f = 3.656207836243727
Optimization restart 4/5, f = 3.656207836328477
Optimization restart 5/5, f = -20.62882771019055
[[20. 25.  5. 30.  3.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -21.71593199379032
Optimization restart 2/5, f = 3.871997132964454
Optimization restart 3/5, f = 3.871997139736994
Optimization restart 4/5, f = -20.48694020038961
Optimization restart 5/5, f = 3.8719971397521906
[[ 3.  25.   5.  70.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1



 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


 Retrying in 1 seconds
Optimization restart 1/5, f = -22.855172845274165
Optimization restart 2/5, f = 3.5962316712748077
Optimization restart 3/5, f = 3.596231673084688
Optimization restart 4/5, f = 3.5962316730846826
Optimization restart 5/5, f = 3.5962316730813217
[[ 3.  8. 27. 30.  1.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -23.886091431685745
Optimization restart 2/5, f = 3.4944466810065684
Optimization restart 3/5, f = -23.886091431332527
Optimization restart 4/5, f = 3.4944466810072257
Optimization restart 5/5, f = 3.4944458300504078
[[20.  8.  5. 70.  3.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -24.402992002574695
Optimization restart 2/5, f = 3.5440843044010304
Optimization restart 3/5, f = -24.402992003135772
Optimization restart 4/5, f = 3.544083794989982
Optimization res

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


 Retrying in 1 seconds
Optimization restart 1/5, f = -25.467718180115313
Optimization restart 2/5, f = 3.236862840409433
Optimization restart 3/5, f = -25.46771818025805
Optimization restart 4/5, f = 3.236863160931364
Optimization restart 5/5, f = 3.236863161000663
[[20.  8.  5. 30.  1.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1



...

Optimization restart 4/5, f = -239.1498236253689
Optimization restart 5/5, f = 38.493894744520304
[[ 9.          8.         15.28880498 70.          1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -241.4718202523697
Optimization restart 2/5, f = -193.1449105642247
Optimization restart 3/5, f = 38.328815119934674
Optimization restart 4/5, f = -241.47183329992396
Optimization restart 5/5, f = -241.4718332988799
[[20.          8.          5.         45.29974782  3.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -243.73667353318922
Optimization restart 2/5, f = -243.73667352251772
Optimization restart 3/5, f = -243.7366735340795
Optimization restart 4/5, f = -243.73667353028952
Optimization restart 5/5, f = 38.41143703641504
[[ 3.          8.         17.01714522 46.

 Retrying in 1 seconds
Optimization restart 1/5, f = -269.56781024291735
Optimization restart 2/5, f = -222.02080303434388
Optimization restart 3/5, f = 42.24174702957998
Optimization restart 4/5, f = -269.5678059773969
Optimization restart 5/5, f = -269.5678102133104
[[ 3.         14.92752003  5.         58.61792706  1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -271.7867167873659
Optimization restart 2/5, f = -271.78671678741944
Optimization restart 3/5, f = -223.90885507186783
Optimization restart 4/5, f = -271.7867167874493
Optimization restart 5/5, f = -271.7867167826543
[[ 3.         25.          9.97408402 44.8224463   3.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -272.55638504199544
Optimization restart 2/5, f = -224.58646360925502
Optimization res

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 5/5, f = 43.191133324300694
[[ 3.          8.          5.         38.96261593  3.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -273.1907050893194
Optimization restart 2/5, f = -228.4279575386841
Optimization restart 3/5, f = -273.190710480237
Optimization restart 4/5, f = 43.28641939640514
Optimization restart 5/5, f = -273.1907104805407
[[ 3. 25. 27. 30.  3.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -275.46512573547744
Optimization restart 2/5, f = -236.37278592837183
Optimization restart 3/5, f = -236.37278531882475
Optimization restart 4/5, f = -275.46512572945664
Optimization restart 5/5, f = -275.4651257348445
[[12.  25.   5.  70.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 secon

 Retrying in 1 seconds
Optimization restart 1/5, f = -313.4807491729466
Optimization restart 2/5, f = -274.29741083006076
Optimization restart 3/5, f = -313.48074917160704
Optimization restart 4/5, f = -313.480749173088
Optimization restart 5/5, f = -274.297459204614
[[ 3.          8.          5.         54.00058821  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -315.8662175969346
Optimization restart 2/5, f = -276.82836404020145
Optimization restart 3/5, f = -276.82836403973886
Optimization restart 4/5, f = 30.677143809173288
Optimization restart 5/5, f = 30.677141378772518
[[11.          8.         27.         38.78460579  1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -318.1927768217626
Optimization restart 2/5, f = -279.0819076632956
Optimization restar

### Experimental Design - Integrated Variance Reduction

#### Model / Emulator (GP)

In [13]:
emulator_ird = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_ird = GPyModelWrapper(emulator_ird, n_restarts=5)
emukit_model_ird.optimize()

Optimization restart 1/5, f = -18.904667569713787
Optimization restart 2/5, f = -16.174154145089986
Optimization restart 3/5, f = -20.62882771018527
Optimization restart 4/5, f = 3.6562078363206503
Optimization restart 5/5, f = -15.276560778342347


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


#### Acquisition Function

In [14]:
integrated_variance_reduction = IntegratedVarianceReduction(model=emukit_model_ird, space=parameter_space)

#### Optimizer

In [15]:
optimizer = GradientAcquisitionOptimizer(parameter_space)

#### Experimental Design Loop

In [16]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ird,
    space = parameter_space,
    acquisition = integrated_variance_reduction,
    acquisition_optimizer = optimizer,
)

In [17]:
ed_loop.run_loop(user_function_time_loss, 200)

Optimization restart 1/5, f = -20.62882771018527
Optimization restart 2/5, f = 3.656207836320588
Optimization restart 3/5, f = 3.6562078363216095
Optimization restart 4/5, f = 3.6562078363205828
Optimization restart 5/5, f = -20.62882771019055
[[17.         23.32748358  8.42152435 33.41587479  2.          2.80209198]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -20.262776246018483
Optimization restart 2/5, f = 3.4885194618037225
Optimization restart 3/5, f = -20.262776238081415
Optimization restart 4/5, f = 3.4885194618067867
Optimization restart 5/5, f = -20.262776246021346
[[11.         13.84125762 24.98554391 30.70943842  2.          4.61993872]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -21.18661453426204
Optimization restart 2/5, f = 3.8797105247745254
Optimization restart 3/5, f = 3.8797105237

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


[[ 3.          8.54120708  5.61644471 64.63306064  2.          2.01050509]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -31.838593320727732
Optimization restart 2/5, f = -31.838593320840523
Optimization restart 3/5, f = 6.944193774624775
Optimization restart 4/5, f = 6.944193774626209
Optimization restart 5/5, f = -31.838593320831876
[[19.         13.02153747  8.31226682 55.77479018  3.          1.7189555 ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -33.61730093405677
Optimization restart 2/5, f = 6.769352259385604
Optimization restart 3/5, f = 6.769352259385595
Optimization restart 4/5, f = 6.769352259385725
Optimization restart 5/5, f = -33.61730093399093
[[18.         10.48662274  9.3891217  32.61277922  3.          3.59998204]]

User function time loss called with 1 inputs to simulate

Evaluatin

 Retrying in 1 seconds
Optimization restart 1/5, f = -52.049969785594755
Optimization restart 2/5, f = 10.808414367992732
Optimization restart 3/5, f = 10.808409918725836
Optimization restart 4/5, f = -52.04996978563721
Optimization restart 5/5, f = 10.808414367154036
[[17.         24.53211668 22.14640013 65.17714762  1.          4.41324722]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -53.67351020735266
Optimization restart 2/5, f = 11.242330128762877
Optimization restart 3/5, f = -53.67351020751498
Optimization restart 4/5, f = 11.242328961174557
Optimization restart 5/5, f = -53.673510206180936
[[20.         17.049019    8.08565913 40.98840518  2.          3.19470958]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -53.35575491756653
Optimization restart 2/5, f = 11.102488269197895
Optimization restar

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


...

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 3/5, f = -119.01733267907011
Optimization restart 4/5, f = -119.01733267910093
Optimization restart 5/5, f = 20.186694110152665
[[ 3.          8.20929269  8.09356448 36.22305852  3.          3.33615426]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -120.64077150506373
Optimization restart 2/5, f = -120.64077150476504
Optimization restart 3/5, f = 20.07414782574145
Optimization restart 4/5, f = -120.64077142142315
Optimization restart 5/5, f = -120.64077150361491
[[11.          9.36944511  6.11162216 49.56981907  3.          4.85436388]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -121.60186228140432
Optimization restart 2/5, f = 20.03343843958468
Optimization restart 3/5, f = 20.033433495143015
Optimization restart 4/5, f = -121.60186228127448


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 5/5, f = -105.93619449252247
[[12.         24.97302654 18.71559563 43.35461848  1.          1.50791223]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -123.72546609243291
Optimization restart 2/5, f = 20.93653820704673
Optimization restart 3/5, f = 20.93653820704901
Optimization restart 4/5, f = 20.93653820704668
Optimization restart 5/5, f = 20.936538206943865
[[ 3.         24.4985083  16.79100605 57.55734744  3.          4.56402474]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -125.89818235886347
Optimization restart 2/5, f = 21.369859180134185
Optimization restart 3/5, f = 21.369857574547638
Optimization restart 4/5, f = 21.369859180134334
Optimization restart 5/5, f = 21.36985918013425
[[ 3.         24.52646004 14.76226604 30.64217829  3.          4.19728835]]

User function tim

KeyboardInterrupt: 

### Random GP

Supervised learning

In [18]:
emulator_rnd = GPy.models.GPRegression(train_X, train_Y, noise_var=1e-10)
emukit_model_rnd = GPyModelWrapper(emulator_rnd, n_restarts=200)
emukit_model_rnd.optimize()
emulator_rnd

Optimization restart 1/200, f = -290.5258497515851
Optimization restart 2/200, f = -359.6234114819579
Optimization restart 3/200, f = 41.695978301154696
Optimization restart 4/200, f = 41.695978302361524
Optimization restart 5/200, f = -359.6234116719032


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 6/200, f = -359.6234116724068
Optimization restart 7/200, f = -359.6234116714769
Optimization restart 8/200, f = -359.62341167201066
Optimization restart 9/200, f = -359.6234116679577
Optimization restart 10/200, f = -359.6234114952532
Optimization restart 11/200, f = -359.62341167244233
Optimization restart 12/200, f = -359.6234116724407
Optimization restart 13/200, f = 41.69597830219246
Optimization restart 14/200, f = 41.69597802016362
Optimization restart 15/200, f = 41.69597830236043
Optimization restart 16/200, f = -359.62341167211093
Optimization restart 17/200, f = -359.62341165274967
Optimization restart 18/200, f = 41.69597785122272
Optimization restart 19/200, f = -359.62341165333225
Optimization restart 20/200, f = -359.6234116718774
Optimization restart 21/200, f = 41.695978301009774
Optimization restart 22/200, f = -359.6234116724237
Optimization restart 23/200, f = -359.623411670319
Optimization restart 24/200, f = 41.69597804718218
Optimization rest

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 32/200, f = -359.6234116704768
Optimization restart 33/200, f = 41.69597825730676
Optimization restart 34/200, f = 41.69597830213169
Optimization restart 35/200, f = -359.62341167170985
Optimization restart 36/200, f = -359.62341167243994
Optimization restart 37/200, f = -359.62341166681324
Optimization restart 38/200, f = 41.69597830236086
Optimization restart 39/200, f = 41.69597830216529
Optimization restart 40/200, f = -359.62341167158854
Optimization restart 41/200, f = 41.69597830236074
Optimization restart 42/200, f = -359.6234116724147
Optimization restart 43/200, f = -359.62341167201714
Optimization restart 44/200, f = -359.62341166934175
Optimization restart 45/200, f = 41.69597830236084
Optimization restart 46/200, f = -359.623411672442
Optimization restart 47/200, f = 41.69597830236077
Optimization restart 48/200, f = 41.69595760462818
Optimization restart 49/200, f = -359.6234116710798
Optimization restart 50/200, f = -359.6234116595541
Optimization re

Optimization restart 190/200, f = -359.623411667635
Optimization restart 191/200, f = -359.6234116724388
Optimization restart 192/200, f = 41.69592581877748
Optimization restart 193/200, f = -359.62341167164846
Optimization restart 194/200, f = -359.6234116700416
Optimization restart 195/200, f = -359.6234116711003
Optimization restart 196/200, f = -359.6234116724385
Optimization restart 197/200, f = 41.69597830236083
Optimization restart 198/200, f = -359.62341167239174
Optimization restart 199/200, f = -359.6234116724402
Optimization restart 200/200, f = -359.62341167171553


GP_regression.,value,constraints,priors
rbf.variance,0.06263445416442809,+ve,
rbf.lengthscale,17.44366051198702,+ve,
Gaussian_noise.variance,0.0006682223302770286,+ve,


### Evaluate

Evaluate RMSE of each model on our test set of 50 randomly selected points.

In [19]:
pred_Y, pred_Y_variance = emukit_model_ed.predict(test_X)
mse_ed = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Model Variance RMSE: ', mse_ed)

Experimental Design w/ Model Variance RMSE:  0.03637841353711442


In [20]:
pred_Y, pred_Y_variance = emukit_model_ird.predict(test_X)
mse_ird = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Integrated Variance Reduction RMSE: ', mse_ird)

Experimental Design w/ Integrated Variance Reduction RMSE:  0.030578350016321143


In [21]:
pred_Y, pred_Y_variance = emukit_model_rnd.predict(test_X)
mse_rnd = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Random GP RMSE: ', mse_rnd)

Random GP RMSE:  0.027845878832201847


In [26]:
p = 1 - (mse_ed/mse_rnd)
f'Experimental Design w/ Model Variance is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Model Variance is -30.642% better than training on random points.'

In [27]:
p = 1 - (mse_ird/mse_rnd)
f'Experimental Design w/ Integrated Variance Reduction is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Integrated Variance Reduction is -9.813% better than training on random points.'

### Save Models

In [28]:
with open(f'experimental_design/models/experimental_design_20_init_points_250_loops_timeloss.pkl', "wb") as f:
     pickle.dump(emukit_model_ed, f)
        
with open(f'experimental_design/models/270_random_points_supervised_GP_timeloss.pkl', "wb") as f:
     pickle.dump(emukit_model_rnd, f)